In [2]:
!pip install boto3
!pip install awscli

  Using cached https://files.pythonhosted.org/packages/68/52/7c0ffae88a765f020e3e85de7d02d64c1b67f8fe78c1d79c5586a493ebf9/boto3-1.9.118-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/0c/2d/67bbe92954c680726b6c1ef1fd57e07fa6d927c049ae7a6c983026926063/botocore-1.12.118-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d7/de/5737f602e22073ecbded7a0c590707085e154e32b68d86545dcc31004c02/s3transfer-0.2.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/49/b1/b6a9ac2a0b91539c50ae60f110d86b80cd802cdb865db935f33f95240880/awscli-1.16.128-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e1/ae/baedc9cb175552e95f3395c43055a6a5e125ae4d48a1d7a924baca83e92e/rsa-3.4.2-py2.py3-none-any.whl


In [4]:
import sklearn.datasets
import sklearn.metrics
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from keras.utils.np_utils import to_categorical
import numpy as np
import pandas as pd
import warnings
from keras.preprocessing.text import Tokenizer
import re
import boto3
import json
import awscli

C:\Users\eason\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
s3 = boto3.resource('s3')
for bucket in s3.buckets.all():
    print(bucket.name)

easonheyc


In [6]:
client = boto3.client(service_name='comprehend', region_name='us-east-2')

In [18]:
response = client.start_sentiment_detection_job(
    InputDataConfig={
        'S3Uri': 's3://easonheyc/data.csv',
        'InputFormat': 'ONE_DOC_PER_LINE'
    },
    OutputDataConfig={
        'S3Uri': 's3://easonheyc/sentiment'
    },
    DataAccessRoleArn='arn:aws:iam::280546296447:role/service-role/AmazonComprehendServiceRole-eason',
    JobName='sentiment',
    LanguageCode='en'
)

In [19]:
print("Start response: %s\n", response)

Start response: %s
 {'JobId': '92ca5102468380f064e74d03f3c15402', 'JobStatus': 'SUBMITTED', 'ResponseMetadata': {'RequestId': '5fc81109-4b73-11e9-b023-3d56d2e7451a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '5fc81109-4b73-11e9-b023-3d56d2e7451a', 'content-type': 'application/x-amz-json-1.1', 'content-length': '68', 'date': 'Thu, 21 Mar 2019 00:50:51 GMT'}, 'RetryAttempts': 0}}


In [37]:
# Check the status of the job
describe_response = client.describe_sentiment_detection_job(JobId=response['JobId'])
print("Describe response: %s\n", describe_response)

Describe response: %s
 {'SentimentDetectionJobProperties': {'JobId': '92ca5102468380f064e74d03f3c15402', 'JobName': 'sentiment', 'JobStatus': 'COMPLETED', 'SubmitTime': datetime.datetime(2019, 3, 20, 20, 50, 51, 219000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2019, 3, 20, 20, 59, 29, 731000, tzinfo=tzlocal()), 'InputDataConfig': {'S3Uri': 's3://easonheyc/data.csv', 'InputFormat': 'ONE_DOC_PER_LINE'}, 'OutputDataConfig': {'S3Uri': 's3://easonheyc/sentiment/280546296447-SENTIMENT-92ca5102468380f064e74d03f3c15402/output/output.tar.gz'}, 'LanguageCode': 'en', 'DataAccessRoleArn': 'arn:aws:iam::280546296447:role/service-role/AmazonComprehendServiceRole-eason'}, 'ResponseMetadata': {'RequestId': 'c9bc4840-4b74-11e9-ad65-3b847a0dc006', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'c9bc4840-4b74-11e9-ad65-3b847a0dc006', 'content-type': 'application/x-amz-json-1.1', 'content-length': '523', 'date': 'Thu, 21 Mar 2019 01:00:58 GMT'}, 'RetryAttempts': 0}}


In [46]:
output = pd.read_csv('output.csv')

In [82]:
output.head(10)

,"{""File"": ""data.csv""","""Line"": 0","""Sentiment"": ""NEUTRAL""","""SentimentScore"": {""Mixed"": 0.01039357390254736","""Negative"": 0.058567967265844345","""Neutral"": 0.8293421864509583","""Positive"": 0.10169627517461777}}"
0,"{""File"": ""data.csv""","""Line"": 1","""Sentiment"": ""POSITIVE""","""SentimentScore"": {""Mixed"": 0.002648639958351...","""Negative"": 0.002754013054072857","""Neutral"": 0.20098717510700226","""Positive"": 0.793610155582428}}"
1,"{""File"": ""data.csv""","""Line"": 2","""Sentiment"": ""NEUTRAL""","""SentimentScore"": {""Mixed"": 0.009224679321050644","""Negative"": 0.00791002158075571","""Neutral"": 0.793932318687439","""Positive"": 0.18893304467201233}}"
2,"{""File"": ""data.csv""","""Line"": 3","""Sentiment"": ""NEUTRAL""","""SentimentScore"": {""Mixed"": 0.006865730043500662","""Negative"": 0.1357971727848053","""Neutral"": 0.8557537794113159","""Positive"": 0.0015832841163501143}}"
3,"{""File"": ""data.csv""","""Line"": 4","""Sentiment"": ""POSITIVE""","""SentimentScore"": {""Mixed"": 0.01740359328687191","""Negative"": 0.0007852203561924398","""Neutral"": 0.2256394475698471","""Positive"": 0.7561717629432678}}"
4,"{""File"": ""data.csv""","""Line"": 5","""Sentiment"": ""NEUTRAL""","""SentimentScore"": {""Mixed"": 0.004719793330878019","""Negative"": 0.03092263825237751","""Neutral"": 0.9173388481140137","""Positive"": 0.047018732875585556}}"
5,"{""File"": ""data.csv""","""Line"": 6","""Sentiment"": ""NEUTRAL""","""SentimentScore"": {""Mixed"": 0.009875085204839706","""Negative"": 0.07467585802078247","""Neutral"": 0.8431217670440674","""Positive"": 0.07232736796140671}}"
6,"{""File"": ""data.csv""","""Line"": 7","""Sentiment"": ""NEUTRAL""","""SentimentScore"": {""Mixed"": 0.002160169649869...","""Negative"": 0.0043444205075502396","""Neutral"": 0.9272862672805786","""Positive"": 0.06620916724205017}}"
7,"{""File"": ""data.csv""","""Line"": 8","""Sentiment"": ""POSITIVE""","""SentimentScore"": {""Mixed"": 0.028535146266222","""Negative"": 0.00031787517946213484","""Neutral"": 0.013152285479009151","""Positive"": 0.9579946994781494}}"
8,"{""File"": ""data.csv""","""Line"": 9","""Sentiment"": ""NEUTRAL""","""SentimentScore"": {""Mixed"": 0.004719793330878019","""Negative"": 0.03092263825237751","""Neutral"": 0.9173388481140137","""Positive"": 0.047018732875585556}}"
9,"{""File"": ""data.csv""","""Line"": 10","""Sentiment"": ""NEUTRAL""","""SentimentScore"": {""Mixed"": 0.002401404781267047","""Negative"": 0.01871020346879959","""Neutral"": 0.9573071599006653","""Positive"": 0.021581171080470085}}"
